In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST-data', one_hot=True, reshape=True)

c:\anaconda3\envs\py36tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [10]:
tf.__version__
tf.reset_default_graph()   # clear all graph

In [11]:
#initial weight and bias
#WARNNING!!! Don't initial zero value into weight and bias
W1 = tf.Variable(name="W1",initial_value=tf.truncated_normal(shape=[784,512],stddev=0.1),dtype=tf.float32)
b1 = tf.Variable(name="b1",initial_value=tf.constant(0.5,shape=[512]),dtype=tf.float32)

W2 = tf.Variable(name="W2",initial_value=tf.truncated_normal(shape=[512,10],stddev=0.1),dtype=tf.float32)
b2 = tf.Variable(name="b2",initial_value=tf.constant(0.5,shape=[10]),dtype=tf.float32)


In [12]:
#mnist Data is a digit image that have dimension is 28*28 pixels.
#create a model

Xdata = tf.placeholder(shape=[None,784],dtype=tf.float32)/255
Ydata = tf.placeholder(shape=[None,10],dtype=tf.float32)

#Hidden Layer
Layer1 = tf.matmul(Xdata,W1)+b1
Layer1 = tf.nn.relu(Layer1)
Layer2 = tf.matmul(Layer1,W2)+b2

#output layer
yout = tf.nn.softmax(Layer2,name="yout")


In [18]:
# defined objective function and Optimizer 
cross_en = -tf.reduce_sum(tf.log(yout)*Ydata,reduction_indices=1)
cross_en = tf.reduce_mean(cross_en,name="cost_tensor")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-4).minimize(cross_en)   #change to AdamOptimizer

In [19]:
#create evaluate tensor 
y_predict = tf.argmax(yout,axis=1)
y_true = tf.argmax(Ydata,axis=1)
correnctPredict = tf.equal(y_predict,y_true)
Acc = tf.reduce_mean(tf.cast(correnctPredict,dtype=tf.float32),name="acc_tensor")

<img src="./imageFile/Tensor2.jpg">
<img src="./imageFile/Tensor1.jpg">

In [ ]:

with tf.Session() as sess:
    # tensorboard write log
    tf_train_board = tf.summary.FileWriter(logdir="./TF2MLP/Train",graph=sess.graph)
    tf_test_board = tf.summary.FileWriter(logdir="./TF2MLP/Test",graph=sess.graph)

    tf.summary.scalar("costValue",cross_en)
    tf.summary.scalar("AccValue",Acc)
    summary_op = tf.summary.merge_all()

    #initial weight
    init = tf.initializers.global_variables()
    sess.run(init)

    for i in range(60000): #100 epoach
        batchX,batchY = mnist.train.next_batch(100)
        summaryValue,_ = sess.run([summary_op,optimizer],feed_dict={Xdata:batchX,Ydata:batchY})
        tf_train_board.add_summary(summaryValue,global_step=i)
        if i%1000 == 0:
            (xTest, yTest) = (mnist.test.images,mnist.test.labels)
            vAC,vCro,summaryValue = sess.run([Acc,cross_en,summary_op],feed_dict={Xdata:xTest,Ydata:yTest})
            print("Accuracy= {:f} Lost= {:f}".format(vAC,vCro))



Accuracy= 0.124300 Lost= 3.041941
Accuracy= 0.925100 Lost= 0.265061
Accuracy= 0.941500 Lost= 0.206554
Accuracy= 0.951600 Lost= 0.172444
